the code below is meant to perform a loop over the webpage and each time scrap data from the table. For some reason not yet figured out, it doesnt work

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import duckdb

pagenum = 0
all_data = []
with requests.Session() as s:
    s.headers = {'User-Agent':'python-requests/2.28.2'}
    requests.session().headers
    while True:
        # URL of the web page containing the HTML table to scrape
        #url = 'https://hfr.health.gov.ng/facilities/hospitals-list?page=' + str(i)
        pagenum+=1
        url = 'https://hfr.health.gov.ng/facilities/hospitals-search?_token=lqDSClMX3WVEbiQnIbJfYQKYUajok3Y1WymK3E2l&state_id=1&lga_id=1&ward_id=0&facility_level_id=0&ownership_id=0&operational_status_id=1&registration_status_id=0&license_status_id=0&geo_codes=0&service_type=0&service_category_id=0&entries_per_page=500&page={pagenum}'
        # Fetch the web page content using requests library
        page = s.get(url).text

        # Create BeautifulSoup object from HTML
        soup = BeautifulSoup(page, 'html.parser')

        # Find the table element in the HTML
        table = soup.find('table')

        # Get the column names from the table header
        headers = []
        for th in table.find_all('th'):
            headers.append(th.text.strip().replace(" ","_").lower())

        # Loop over each row in the table body and store the data in a list of dictionaries
        data = []
        for tr in table.find_all('tr'):
            row = {}
            for i, td in enumerate(tr.find_all('td')):
                row[headers[i]] = td.text.strip()
            if row:
                data.append(row)
        
        # Append the data from the current page to the list of all data
        all_data.extend(data)

        # Find the next page link (if any)
        next_link = soup.find('a', {'class': 'page-link'},True)
   
    
        if next_link:
            # If there is a next link, update the URL and continue to next page
            url = next_link['href']
        else:
            # If there is no next link, we're done scraping
            break
   
# Create pandas DataFrame from the list of dictionaries
df = pd.DataFrame(all_data)
print(df)

#df.to_excel("bystate.xlsx",sheet_name="Hospital number", index=False)

code showing to create new columns on a dataframe

In [ ]:

import pandas as pd
import numpy as np

# create a dictionary of films
films = {
    "Id": [1,2,3,5,6],
    "Title": ["Jurassic Park", "Spider-man", "King-Kong", "Superman Returns","Titanic"],
    "Released Date": ["1993-06-11","2002-05-03","2005-12-14","2006-07-14","1998-01-23"],
    "Run Time": [126, 121, 187,154, 194],
    "Genre": ["Adventure", "Action", "Adventure", "Action", "Romance"]
}

# create a Pandas dataframe based upon this
df = pd.DataFrame(films)

# lenght of film title (the new column on the left = some calculation done on the old df)
df["Title length"] = df["Title"].str.len()

# hours and minutes
df["Hours"] = np.floor(df["Run Time"] / 60).astype(int)
df["Minutes"] = df["Run Time"] % 60

# convert the "date" to a real date
df["RealDate"] = pd.to_datetime(df["Released Date"])
df["RelYear"] = pd.DatetimeIndex(df["RealDate"]).year

# print this out
print("\n")
print(df)
print("\n")
print(df.dtypes)

Code to parse a HTML table into a pandas dataframe

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL of the website and the filter parameter
url = 'https://hfr.health.gov.ng/facilities/hospitals-list'
params = {'filter': 'Tertiary'}

# Send a GET request to the website with the filter parameter
response = requests.get(url, params=params)

# Parse the HTML content of the response using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table on the website using its HTML tag and class
table = soup.find('table')

# Convert the table into a Pandas DataFrame
df = pd.read_html(str(table))[0]
print(df)
# Save the DataFrame to an Excel workbook
df.to_excel('output.xlsx', index=False)